# __Fetch aoe2recs.com match details parallely__

### __Load libraries__

In [1]:
from pymongo import MongoClient

from tqdm import tqdm

from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import os
import random

from requests_ip_rotator import ApiGateway
import requests
import mechanize
import http.cookiejar
import random
import warnings

import multiprocessing
from joblib import Parallel, delayed

cores =  20 # API rate limiter forces to use 20
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### __Source functions__

In [2]:
%run aoe2rec_api_modules.ipynb

### __Fetch proxies & user agents__

In [3]:
proxy_list = fetch_proxies(https = False)
uas_list = fetch_user_agents()

### __Set up mongo connection__

In [4]:
db_conn = setup_mongo_conn()

### __Delete invalid matches__

In [5]:
delete_invalid_match_mongo(db_conn)

Invalid entries deleted !


### __Fetch match details__

In [ ]:
# Choose between fetching new matches or missing ones
# Optionally delete bad records - id = -999

for index, row in proxy_list.iterrows():
    
    ip = 'http://' + row['IP Address'] + ':' + str(row['Port'])
    proxy = {'http' : ip}
    ua = uas_list.sample()['User agent'].iloc[0]
    
    print(ip + " -- " + ua)
    
    try:
        fetch_matches_iterate_aoe2recs(db_conn, collect = cores, type = 'new', clean_empty = False, 
                                       cores = cores, proxy = proxy, ua = ua, progress_bar = False)
    except:
        print('Skipping to next IP !')
        pass